In [1]:
from google.cloud import bigquery
import pandas as pd
import altair as alt
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/yigityavuzyildiz/Desktop/bruin_case/bruin-hiring-1c80ac56a1b8.json"

client = bigquery.Client()

In [48]:
def run_query_and_visualize(query, x_column, y_column, chart_type='bar', title=''):
    """
    Runs a BigQuery SQL query and visualizes the result using Altair.

    Parameters:
    query (str): The SQL query to run.
    x_column (str): The column to use for the x-axis.
    y_column (str): The column to use for the y-axis.
    chart_type (str): The type of chart to create ('bar', 'line'). Default is 'bar'.
    title (str): The title of the chart.
    
    Returns:
    None
    """
    
    query_job = client.query(query)  
    df = query_job.to_dataframe()

    print(df)

    if df[x_column].dtype == 'datetime64[ns]':
        df[x_column] = df[x_column].dt.strftime('%Y-%m-%d')

    selection = alt.selection_multi(fields=[x_column], bind='legend')

    if chart_type == 'bar':
        chart = alt.Chart(df).mark_bar().encode(
            x=alt.X(x_column, sort='-y'),
            y=y_column,
            tooltip=[x_column, y_column],
            color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray'))
        ).properties(
            title=title
        ).add_selection(
            selection
        ).interactive()
    elif chart_type == 'line':
        chart = alt.Chart(df).mark_line().encode(
            x=x_column,
            y=y_column,
            tooltip=[x_column, y_column]
        ).properties(
            title=title
        ).interactive()
    else:
        raise ValueError("Unsupported chart type: {}".format(chart_type))

    chart.show()


# 1) Most Popular Repositories


In [32]:
query = """
SELECT repo_name, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY repo_name
ORDER BY commit_count DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'repo_name', 'commit_count', chart_type='bar', title='Most Popular Repositories')


               repo_name  commit_count
0         torvalds/linux        602604
1            apple/swift         38891
2         twbs/bootstrap         11777
3       Microsoft/vscode          6909
4         facebook/react          6875
5  tensorflow/tensorflow          5253


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 2) Top Contributors


In [33]:
query = """
SELECT author.name as author_name, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY author_name
ORDER BY commit_count DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'author_name', 'commit_count', chart_type='bar', title='Top Contributors')


             author_name  commit_count
0         Linus Torvalds         20776
1        David S. Miller          7552
2             Mark Brown          6466
3      H Hartley Sweeten          5857
4              Mark Otto          5823
5           Takashi Iwai          5801
6                Al Viro          5439
7            Ingo Molnar          4858
8          Chris Lattner          4448
9  Mauro Carvalho Chehab          4043


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 3) Developers Contributing to Multiple Repositories


In [34]:
query = """
SELECT author.name as author_name, COUNT(DISTINCT repo_name) as repo_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY author_name
HAVING repo_count > 1
ORDER BY repo_count DESC;
"""
run_query_and_visualize(query, 'author_name', 'repo_count', chart_type='bar', title='Developers Contributing to Multiple Repositories')


       author_name  repo_count
0             Alex           3
1       Henry Wong           3
2     Prayag Verma           3
3       Alex Smith           2
4             Dave           2
..             ...         ...
66         Michael           2
67            Nick           2
68  Pascal Borreli           2
69      Tom Parker           2
70           Undo1           2

[71 rows x 2 columns]


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 4) Most Popular Email Domains


In [35]:
query = """
SELECT SPLIT(author.email, '@')[SAFE_OFFSET(1)] as domain, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY domain
ORDER BY commit_count DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'domain', 'commit_count', chart_type='bar', title='Most Popular Email Domains')


                 domain  commit_count
0             gmail.com         69448
1            redhat.com         34908
2             apple.com         33919
3             intel.com         33910
4  linux-foundation.org         18684
5               suse.de         14575
6            linaro.org         13556
7                ti.com         10981
8           samsung.com         10682
9       linux.intel.com         10241


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 5) Number of Commits Per Day


In [47]:
query = """
SELECT DATE(committer.date) as commit_date, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY commit_date
ORDER BY commit_date;
"""
run_query_and_visualize(query, 'commit_date', 'commit_count', chart_type='line', title='Number of Commits Per Day')


     commit_date  commit_count
0     2005-04-16           199
1     2005-04-17            21
2     2005-04-18            39
3     2005-04-19            70
4     2005-04-20            11
...          ...           ...
4064  2016-06-18            43
4065  2016-06-19            58
4066  2016-06-20           113
4067  2016-06-21           142
4068  2016-06-22           108

[4069 rows x 2 columns]


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


TypeError: Object of type date is not JSON serializable

alt.Chart(...)

# 6) Number of Commits Per Weekday


In [37]:
query = """
SELECT FORMAT_DATE('%A', DATE(committer.date)) as weekday, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY weekday
ORDER BY commit_count DESC;
"""
run_query_and_visualize(query, 'weekday', 'commit_count', chart_type='bar', title='Number of Commits Per Weekday')


     weekday  commit_count
0    Tuesday        121674
1  Wednesday        120722
2     Monday        115061
3   Thursday        114669
4     Friday        112297
5   Saturday         44814
6     Sunday         43072


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 7) Additional Insights

### 1. Commits by Hour of the Day

In [38]:
query = """
SELECT EXTRACT(HOUR FROM committer.date) as commit_hour, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY commit_hour
ORDER BY commit_hour;
"""
run_query_and_visualize(query, 'commit_hour', 'commit_count', chart_type='bar', title='Commits by Hour of the Day')


    commit_hour  commit_count
0             0         29068
1             1         25508
2             2         19505
3             3         19028
4             4         18636
5             5         18989
6             6         19102
7             7         20077
8             8         23389
9             9         23524
10           10         21813
11           11         20422
12           12         22136
13           13         22654
14           14         25652
15           15         34674
16           16         35668
17           17         36183
18           18         39444
19           19         40465
20           20         42089
21           21         38247
22           22         37062
23           23         38974


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

### 2. Most Active Authors by Number of Repositories Contributed To


In [40]:
query = """
SELECT author.name as author_name, COUNT(DISTINCT repo_name) as repo_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY author_name
ORDER BY repo_count DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'author_name', 'repo_count', chart_type='bar', title='Most Active Authors by Number of Repositories Contributed To')


       author_name  repo_count
0             Alex           3
1     Prayag Verma           3
2       Henry Wong           3
3             Mark           2
4  Anton Blanchard           2
5      John Wright           2
6    Jody McIntyre           2
7  Max F. Albrecht           2
8             Matt           2
9      Kevin Huang           2


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

### 3. Number of Files Changed per Commit


In [41]:
query = """
SELECT commit, ARRAY_LENGTH(difference) as files_changed
FROM `bruin-hiring.github_repos.sample_commits`
ORDER BY files_changed DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'commit', 'files_changed', chart_type='bar', title='Number of Files Changed per Commit')


                                     commit  files_changed
0  e5451c8f8330e03ad3cfa16048b4daf961af434f          53003
1  f43dc23d5ea91fca257be02138a255f02d98e806          26947
2  2d8ad8719591fa803b0d589ed057fa46f49b7155          23633
3  cf9ce948f47640797bd19980e1d99c6d17d0bdc3          21526
4  e8cdfb0509f48d44d95d68d4f42d8d71a9ba4acd          19597
5  9938b04472d5c59f8bd8152a548533a8599596a2          18852
6  b74b953b998bcc2db91b694446f3a2619ec32de6          18512
7  7a9787e1eba95a166265e6a260cf30af04ef0a99          18502
8  94bd217e2d683719ab21a4ac117d8a1b91cbedc9          18420
9  b2aaf8f74cdc84a9182f6cabf198b7763bcb9d40          17815


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

### 4. Commits by Author and Repository


In [43]:
query = """
SELECT author.name as author_name, repo_name, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`
GROUP BY author_name, repo_name
ORDER BY commit_count DESC
LIMIT 10;
"""
run_query_and_visualize(query, 'author_name', 'commit_count', chart_type='bar', title='Commits by Author and Repository')


             author_name       repo_name  commit_count
0         Linus Torvalds  torvalds/linux         20776
1        David S. Miller  torvalds/linux          7552
2             Mark Brown  torvalds/linux          6466
3      H Hartley Sweeten  torvalds/linux          5857
4              Mark Otto  twbs/bootstrap          5823
5           Takashi Iwai  torvalds/linux          5801
6                Al Viro  torvalds/linux          5439
7            Ingo Molnar  torvalds/linux          4858
8          Chris Lattner     apple/swift          4448
9  Mauro Carvalho Chehab  torvalds/linux          4043


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

# 8) Bonus

In [44]:
query = """
SELECT repo_name, diff.new_path as file_path, COUNT(*) as commit_count
FROM `bruin-hiring.github_repos.sample_commits`,
UNNEST(difference) as diff
GROUP BY repo_name, file_path
ORDER BY commit_count DESC
LIMIT 5;
"""

run_query_and_visualize(query, 'file_path', 'commit_count', chart_type='bar', title='Top 5 Files with the Most Commits Per Repo')


        repo_name                             file_path  commit_count
0  torvalds/linux                           MAINTAINERS         10465
1  torvalds/linux  drivers/gpu/drm/i915/intel_display.c          5349
2  torvalds/linux                              Makefile          4994
3  torvalds/linux         sound/pci/hda/patch_realtek.c          4668
4  torvalds/linux                      arch/arm/Kconfig          4277


/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yigityavuzyildiz/anaconda3/envs/bruin/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)